In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import copy
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
filepath = '/Users/dangoldberg/Desktop/DataScienceInterviewProjects/Toronto Blue Jays/catcherframe.csv'

In [3]:
df = pd.read_csv(filepath)

# Issues with the dataset

## 1. NaN ump_hp

In [6]:
df[df['ump_hp'].isna()].sample(10)

,parkid,bluejaysgameid,gamepitchsequence,half,balls,strikes,pitcherid,pitcherhand,batterid,batside,...,calledstrike,ball,swings,takes,pitchout,intentball,inplay,pitcheventtype,hbp,ump_hp
305978,496,6AE9DC51-446A-4713-94E8-45AD0F5268E5,56,1,0,1,3603,R,163551,R,...,0,1,0,1,0,0,0,ball,0,NaN
158885,249,EB943949-79EE-41A3-856D-565159BFF374,156,1,1,2,184470,R,3855,R,...,0,1,0,1,0,0,0,ball,0,NaN
232467,387,9141702A-4CCC-45D4-9C14-85A04F0A492F,94,1,0,0,6122,R,10265,R,...,0,1,0,1,0,0,0,ball,0,NaN
67970,70,3BB509AB-A14F-4F82-81DE-8F0C8546040F,105,1,1,1,9826,R,9222,R,...,0,1,0,1,0,0,0,ball,0,NaN
19272,22,8B0C6D34-7525-428F-97EB-48DE8289FADD,154,0,0,0,3853,R,8896,R,...,1,0,0,1,0,0,0,called_strike,0,NaN
198046,299,461F6724-282F-4848-AF74-84B892B85FBC,163,1,0,0,187052,R,4776,L,...,0,0,1,0,0,0,0,swinging_strike,0,NaN
257070,399,ECFBC7B4-76F7-4B7C-BB60-CCAA56CF9E50,99,0,1,0,3792,R,165613,R,...,0,1,0,1,0,0,0,ball,0,NaN
262792,424,413669B0-8738-44E1-9054-35D324F02919,128,0,1,1,6123,R,11133,L,...,0,0,1,0,0,0,0,swinging_strike,0,NaN
258453,399,2DF7B6CD-A1C0-4028-A531-3AEF68373DD3,151,1,0,1,9402,R,161193,R,...,0,0,1,0,0,0,1,hit_into_play_no_out,0,NaN
7045,5,EE0F1DAD-7952-4EF2-A3D7-8F2E1D28E32C,277,0,1,2,14163,R,10540,L,...,0,0,1,0,0,0,0,foul,0,NaN


## 2. batterid sequencing

In [7]:
df.loc[56184-5:56184+5]

,parkid,bluejaysgameid,gamepitchsequence,half,balls,strikes,pitcherid,pitcherhand,batterid,batside,...,calledstrike,ball,swings,takes,pitchout,intentball,inplay,pitcheventtype,hbp,ump_hp
56179,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,199,0,1,1,10006,R,4028,R,...,1,0,0,1,0,0,0,called_strike,0,NaN
56180,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,200,1,1,2,161209,L,150191,L,...,1,0,0,1,0,0,0,called_strike,0,NaN
56181,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,201,0,1,2,10006,R,4028,R,...,0,1,0,1,0,0,0,ball,0,NaN
56182,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,202,1,2,2,161209,L,150191,L,...,0,1,0,1,0,0,0,ball,0,NaN
56183,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,203,0,3,2,10006,R,4028,R,...,0,1,0,1,0,0,0,ball,0,NaN
56184,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,204,1,4,2,161209,L,150191,L,...,0,1,0,1,0,0,0,ball,0,NaN
56185,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,205,0,5,2,10006,R,4028,R,...,0,0,1,0,0,0,1,hit_into_play_out,0,NaN
56186,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,206,1,5,2,161209,L,150191,L,...,1,0,0,1,0,0,0,called_strike,0,NaN
56187,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,207,1,5,2,161209,L,150191,L,...,0,0,1,0,0,0,0,swinging_strike_blocked,0,NaN
56188,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,208,0,0,0,10006,R,6515,L,...,0,0,1,0,0,0,0,swinging_strike,0,NaN


## 3. count with balls > 3

In [10]:
df[df['balls'] > 3].sample(10)

,parkid,bluejaysgameid,gamepitchsequence,half,balls,strikes,pitcherid,pitcherhand,batterid,batside,...,calledstrike,ball,swings,takes,pitchout,intentball,inplay,pitcheventtype,hbp,ump_hp
56375,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,395,0,4,2,3979,R,10511,R,...,0,0,1,0,0,0,0,swinging_strike,0,NaN
56185,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,205,0,5,2,10006,R,4028,R,...,0,0,1,0,0,0,1,hit_into_play_out,0,NaN
56377,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,397,0,4,2,3979,R,10511,R,...,0,0,1,0,0,0,1,hit_into_play,0,NaN
56418,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,438,0,4,2,3874,R,4028,R,...,0,1,0,1,0,0,0,ball,0,NaN
56340,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,360,1,5,2,161878,R,8504,R,...,0,0,1,0,0,0,1,hit_into_play_out,0,NaN
56341,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,361,1,5,2,161209,L,6498,R,...,0,0,1,0,0,0,1,hit_into_play,0,NaN
56064,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,28,0,4,2,10006,R,4028,R,...,0,1,0,1,0,0,0,ball,0,NaN
56420,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,440,0,6,2,3874,R,6515,L,...,0,0,1,0,0,0,0,swinging_strike,0,NaN
56250,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,270,0,5,2,10006,R,10511,R,...,0,0,1,0,0,0,0,foul,0,NaN
56168,67,0AE790E4-46EE-4EF0-95B1-2B0640E6E744,188,1,4,2,161209,L,8504,R,...,0,0,1,0,0,0,0,foul,0,NaN
